In [1]:
from data_utils import DPMDataModule

from ast import literal_eval

import torch
import torch.nn as nn
import torch.optim as optim

from pytorch_lightning import LightningModule
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torchmetrics.functional import accuracy

from transformers import  AutoModelForSequenceClassification

train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
  print('CUDA is not available. Training on CPU ...')
else:
  print('CUDA is available. Training on GPU ...')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CUDA is available. Training on GPU ...


In [2]:
class DontPatronizeMe(LightningModule):
    def __init__(self):
        super(DontPatronizeMe,self).__init__()

        self.n_classes = 7
        self.lr = 0.001

        self.bert = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain").bert
        self.dropout = nn.Dropout(0.5)
        self.linear1 = nn.Linear(768, 768)
        self.linear2 = nn.Linear(768, self.n_classes)
        self.relu = nn.ReLU()

        for param in self.bert.parameters():
            param.require_grad = False

        self.criterion = nn.BCEWithLogitsLoss()


    def forward(self, input_id, mask):
        x = self.bert(input_ids=input_id, attention_mask=mask, return_dict=False)
        x = x[0]
        x = x[:,0]
        x = self.dropout(x)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        return x

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.lr)
        return {
            "optimizer": optimizer,
        }

    def training_step(self, batch, _):
        ids, mask, labels = batch["ids"], batch["mask"], batch["labels"]
        output = self(ids, mask)
        loss = self.criterion(output, labels)
        self.log("train/loss", loss, prog_bar=True, on_epoch=True, on_step=False)
        return loss

    def validation_step(self, batch, _):
        ids, mask, labels = batch["ids"], batch["mask"], batch["labels"]
        output = self(ids, mask)
        loss   = self.criterion(output, labels)
        acc    = accuracy(output, labels.int(), multiclass=True)
        return {"loss": loss, "acc": acc}

    def validation_epoch_end(self, out):
        loss = torch.stack([x["loss"] for x in out]).mean()
        self.log("val/val_loss", loss, on_epoch=True, on_step=False)
        acc = torch.stack([x["acc"] for x in out]).mean()
        self.log("val/val_acc", acc, on_epoch=True, on_step=False)

In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

BATCH_SIZE = 2
EPOCHS = 10000

data = DPMDataModule(num_workers=0, batch_size=4, shuffle=False)
model_name = "bert"
model = DontPatronizeMe()
logger = TensorBoardLogger("tb_logs", name=f"{model_name}")

trainer = Trainer(
    detect_anomaly=True,
    gpus=1,
    enable_model_summary=True,
    logger=logger,
    log_every_n_steps=BATCH_SIZE,
    max_epochs=EPOCHS,

    callbacks=[
        ModelCheckpoint(
            monitor="val/val_loss",
            mode="min",
            dirpath=f"models/{model_name}",
            filename="radar-epoch{epoch:02d}-val_loss{val/val_loss:.2f}",
            auto_insert_metric_name=False,
        ),
        EarlyStopping(monitor="val/val_loss", patience=6),
    ],
)
trainer.fit(model, data)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | bert      | BertModel         | 109 M 
1 | dropout   | Dropout           | 0     
2 | linear1   | Linear            | 590 K 
3 | linear2   | Linear            | 5.4 K 
4 | relu      | ReLU              | 0     
5 | criterion | BCEWithLogitsLoss | 0     
------------------------------------------------
110 M     Trainable params
0         Non-trainable params
110 M     Total params
440.313   Total estimated model params size (MB)
C:\Programs\Anaconda\envs\pytorch\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:631: UserWarning: Checkpoint directory C:\Users\radus\OneDrive - unibuc.ro\Master\An 2\Semestrul 1\NLP2\Project\dontpatronizeme-Semeval2022\task2\models\bert exists and is not empty.
  rank_zero_warn(f"Checkpoint direct

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Programs\Anaconda\envs\pytorch\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


C:\Programs\Anaconda\envs\pytorch\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 2:  20%|█▉        | 49/249 [01:04<04:22,  1.31s/it, loss=0.693, v_num=1, train/loss=0.693] 